---
title: 'Compare Spending: Promo-Heavy vs. Promo-Light'
---
# 7. Compare Spending: Promo-Heavy vs. Promo-Light

**Critical Question:** Do promotion-heavy households spend MORE or LESS than promotion-light households?

- If MORE → They're "smart shoppers" worth retaining
- If LESS → They might be "margin killers"

We'll compare:
- Total annual spending
- Average basket value
- Average price per unit (if available)


In [ ]:

promo_summary = (
    households
    .groupby('promo_segment')
    .agg(
        num_households=('household_id', 'nunique'),
        mean_total_sales=('total_sales', 'mean'),
        median_total_sales=('total_sales', 'median'),
        mean_num_trips=('num_trips', 'mean'),
        mean_avg_basket=('avg_basket_value', 'mean'),
        median_avg_basket=('avg_basket_value', 'median'),
    )
)

if 'avg_price_per_unit' in households.columns:
    promo_summary['mean_avg_price_unit'] = (
        households.groupby('promo_segment')['avg_price_per_unit'].mean()
    )
    promo_summary['median_avg_price_unit'] = (
        households.groupby('promo_segment')['avg_price_per_unit'].median()
    )

promo_summary = promo_summary.round(2)

print("\n" + "=" * 80)
print("SPENDING COMPARISON: Promo-Heavy vs. Promo-Light")
display(promo_summary)



In [ ]:

print("\n" + "=" * 80)
print("STATISTICAL SIGNIFICANCE TESTS")

heavy_baskets = households[households['promo_segment'] == 'promo_heavy']['avg_basket_value'].dropna()
light_baskets = households[households['promo_segment'] == 'promo_light']['avg_basket_value'].dropna()

t_stat_basket, p_value_basket = stats.ttest_ind(heavy_baskets, light_baskets)

print("\n1. Average Basket Value:")
ttest_df = pd.DataFrame({
    'Metric': [
        'Promo-Heavy Mean',
        'Promo-Light Mean',
        'Difference',
        'T-Statistic',
        'P-Value'
    ],
    'Value': [
        f'${heavy_baskets.mean():.2f}',
        f'${light_baskets.mean():.2f}',
        f'${heavy_baskets.mean() - light_baskets.mean():.2f}',
        f'{t_stat_basket:.4f}',
        f'{p_value_basket:.6f}'
    ]
})
display(ttest_df.set_index('Metric'))

if p_value_basket < 0.05:
    direction = "higher" if heavy_baskets.mean() > light_baskets.mean() else "lower"
    print(f"   ✓ Promo-heavy households have statistically significantly {direction} basket values (p < 0.05)")
else:
    print("   ✗ No statistically significant difference (p >= 0.05)")

heavy_spending = households[households['promo_segment'] == 'promo_heavy']['total_sales'].dropna()
light_spending = households[households['promo_segment'] == 'promo_light']['total_sales'].dropna()

t_stat_sales, p_value_sales = stats.ttest_ind(heavy_spending, light_spending)

print("\n2. Total Annual Spending:")
print(f"   Promo-heavy mean: ${heavy_spending.mean():.2f}")
print(f"   Promo-light mean: ${light_spending.mean():.2f}")
print(f"   Difference: ${heavy_spending.mean() - light_spending.mean():.2f}")
print(f"   T-statistic: {t_stat_sales:.4f}")
print(f"   P-value: {p_value_sales:.6f}")

if p_value_sales < 0.05:
    direction = "higher" if heavy_spending.mean() > light_spending.mean() else "lower"
    print(f"   ✓ Promo-heavy households have statistically significantly {direction} total spending (p < 0.05)")
else:
    print("   ✗ No statistically significant difference (p >= 0.05)")

if 'avg_price_per_unit' in households.columns:
    heavy_prices = households[households['promo_segment'] == 'promo_heavy']['avg_price_per_unit'].dropna()
    light_prices = households[households['promo_segment'] == 'promo_light']['avg_price_per_unit'].dropna()
    
    t_stat_price, p_value_price = stats.ttest_ind(heavy_prices, light_prices)
    
    print("\n3. Average Price Per Unit:")
    print(f"   Promo-heavy mean: ${heavy_prices.mean():.2f}")
    print(f"   Promo-light mean: ${light_prices.mean():.2f}")
    print(f"   Difference: ${heavy_prices.mean() - light_prices.mean():.2f}")
    print(f"   T-statistic: {t_stat_price:.4f}")
    print(f"   P-value: {p_value_price:.6f}")
    
    if p_value_price < 0.05:
        direction = "higher" if heavy_prices.mean() > light_prices.mean() else "lower"
        print(f"   ✓ Promo-heavy households buy statistically significantly {direction} priced items (p < 0.05)")
    else:
        print("   ✗ No statistically significant difference (p >= 0.05)")



In [ ]:

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

colors = ['#27ae60', '#e74c3c']

ax1 = axes[0]
basket_data = promo_summary['mean_avg_basket']
bars1 = ax1.bar(basket_data.index, basket_data.values, color=colors)
ax1.set_ylabel('Average Basket Value ($)', fontsize=12, fontweight='bold')
ax1.set_title('Average Basket Value by Segment', fontsize=13, fontweight='bold')
ax1.set_xlabel('Promotion Segment', fontsize=11)
ax1.tick_params(axis='x', rotation=0)
ax1.grid(axis='y', alpha=0.3)

for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'${height:.2f}',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

ax2 = axes[1]
sales_data = promo_summary['mean_total_sales']
bars2 = ax2.bar(sales_data.index, sales_data.values, color=colors)
ax2.set_ylabel('Total Annual Spending ($)', fontsize=12, fontweight='bold')
ax2.set_title('Annual Spending by Segment', fontsize=13, fontweight='bold')
ax2.set_xlabel('Promotion Segment', fontsize=11)
ax2.tick_params(axis='x', rotation=0)
ax2.grid(axis='y', alpha=0.3)

for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'${height:.0f}',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

ax3 = axes[2]
if 'mean_avg_price_unit' in promo_summary.columns:
    price_data = promo_summary['mean_avg_price_unit']
    bars3 = ax3.bar(price_data.index, price_data.values, color=colors)
    ax3.set_ylabel('Avg Price Per Unit ($)', fontsize=12, fontweight='bold')
    ax3.set_title('Item Prices by Segment', fontsize=13, fontweight='bold')
    label_format = '${:.2f}'
else:
    trips_data = promo_summary['mean_num_trips']
    bars3 = ax3.bar(trips_data.index, trips_data.values, color=colors)
    ax3.set_ylabel('Avg Shopping Trips', fontsize=12, fontweight='bold')
    ax3.set_title('Shopping Frequency by Segment', fontsize=13, fontweight='bold')
    label_format = '{:.1f}'

ax3.set_xlabel('Promotion Segment', fontsize=11)
ax3.tick_params(axis='x', rotation=0)
ax3.grid(axis='y', alpha=0.3)

for bar in bars3:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
             label_format.format(height),
             ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()

plt.show()



In [ ]:
print("SEGMENTATION DIAGNOSTIC")

print("\n1. Promo segment distribution:")
print(households["promo_segment"].value_counts())

print("\n2. Threshold values:")
print(
    f"   Discount threshold (75th percentile): {households['discount_share'].quantile(0.75):.1%}"
)
print(
    f"   Coupon threshold (75th percentile): {households['coupons_per_100_items'].quantile(0.75):.2f}"
)

print("\n3. How many households meet each criterion:")
print(
    f"   High discount users: {(households['discount_share'] >= households['discount_share'].quantile(0.75)).sum()}"
)
print(
    f"   High coupon users: {(households['coupons_per_100_items'] >= households['coupons_per_100_items'].quantile(0.75)).sum()}"
)

print("\n4. Sample of households with their segments:")
display(
    households[
        ["household_id", "discount_share", "coupons_per_100_items", "promo_segment"]
    ].head(20)
)

print("\n5. Promo summary contents:")
display(promo_summary)


In [ ]:

print("\n" + "=" * 80)
print("KEY FINDING: PROMO-HEAVY VS. PROMO-LIGHT")


promo_segments = promo_summary.index.tolist()

try:
    heavy_idx = [i for i, s in enumerate(promo_segments) if "heavy" in str(s).lower()][
        0
    ]
    light_idx = [i for i, s in enumerate(promo_segments) if "light" in str(s).lower()][
        0
    ]

    segment_heavy = promo_segments[heavy_idx]
    segment_light = promo_segments[light_idx]


except IndexError:
    print("⚠️ Using position-based indexing")
    heavy_idx = 0
    light_idx = 1
    segment_heavy = promo_segments[0]
    segment_light = promo_segments[1]

diff_basket = (
    promo_summary.iloc[heavy_idx]["mean_avg_basket"]
    - promo_summary.iloc[light_idx]["mean_avg_basket"]
)
diff_pct_basket = (diff_basket / promo_summary.iloc[light_idx]["mean_avg_basket"]) * 100

diff_sales = (
    promo_summary.iloc[heavy_idx]["mean_total_sales"]
    - promo_summary.iloc[light_idx]["mean_total_sales"]
)
diff_pct_sales = (diff_sales / promo_summary.iloc[light_idx]["mean_total_sales"]) * 100

print("\n📊 Basket Value:")
if diff_basket > 0:
    print(
        f"   {segment_heavy} households have {diff_pct_basket:.1f}% HIGHER average basket values"
    )
    print(
        f"   (${promo_summary.iloc[heavy_idx]['mean_avg_basket']:.2f} vs ${promo_summary.iloc[light_idx]['mean_avg_basket']:.2f})"
    )
    print("   ✓ This suggests they're high-value customers, not margin killers!")
else:
    print(
        f"   {segment_heavy} households have {abs(diff_pct_basket):.1f}% LOWER average basket values"
    )
    print(
        f"   (${promo_summary.iloc[heavy_idx]['mean_avg_basket']:.2f} vs ${promo_summary.iloc[light_idx]['mean_avg_basket']:.2f})"
    )
    print(
        "   ⚠️ This suggests promotions may attract price-sensitive, low-margin shoppers"
    )

print("\n📊 Annual Spending:")
if diff_sales > 0:
    print(f"   {segment_heavy} households spend {diff_pct_sales:.1f}% MORE annually")
    print(
        f"   (${promo_summary.iloc[heavy_idx]['mean_total_sales']:.2f} vs ${promo_summary.iloc[light_idx]['mean_total_sales']:.2f})"
    )
else:
    print(
        f"   {segment_heavy} households spend {abs(diff_pct_sales):.1f}% LESS annually"
    )
    print(
        f"   (${promo_summary.iloc[heavy_idx]['mean_total_sales']:.2f} vs ${promo_summary.iloc[light_idx]['mean_total_sales']:.2f})"
    )

if "mean_avg_price_unit" in promo_summary.columns:
    diff_price = (
        promo_summary.iloc[heavy_idx]["mean_avg_price_unit"]
        - promo_summary.iloc[light_idx]["mean_avg_price_unit"]
    )
    diff_pct_price = (
        diff_price / promo_summary.iloc[light_idx]["mean_avg_price_unit"]
    ) * 100

    print("\n📊 Item Prices:")
    if diff_price > 0:
        print(
            f"   {segment_heavy} households buy {diff_pct_price:.1f}% more expensive items"
        )
        print(
            f"   (${promo_summary.iloc[heavy_idx]['mean_avg_price_unit']:.2f} vs ${promo_summary.iloc[light_idx]['mean_avg_price_unit']:.2f} per unit)"
        )
    else:
        print(
            f"   {segment_heavy} households buy {abs(diff_pct_price):.1f}% cheaper items"
        )
        print(
            f"   (${promo_summary.iloc[heavy_idx]['mean_avg_price_unit']:.2f} vs ${promo_summary.iloc[light_idx]['mean_avg_price_unit']:.2f} per unit)"
        )


## 8. Demographic Analysis of Promotion Users

Now we examine **who** the promotion-heavy households are. We'll analyze:

- **Household composition** (kids vs. no kids)
- **Income brackets**
- **Household size**
- **Age groups**

This helps us understand which segments to target with promotional campaigns.


In [ ]:


if 'kids_count' in households.columns:
    households['has_kids'] = (households['kids_count'] != '0').astype(int)
    
    kids_promo = (
        households
        .groupby(['has_kids', 'promo_segment'])
        .size()
        .unstack(fill_value=0)
    )
    
    kids_promo['total'] = kids_promo.sum(axis=1)
    kids_promo['promo_heavy_share'] = (
        kids_promo['promo_heavy'] / kids_promo['total']
    )
    
    print("\nPromotion segment distribution by kids status:")
    display(kids_promo)
    
    kids_spend = (
        households
        .groupby(['has_kids', 'promo_segment'])
        .agg(
            num_households=('household_id', 'nunique'),
            mean_total_sales=('total_sales', 'mean'),
            mean_avg_basket=('avg_basket_value', 'mean'),
            mean_discount_share=('discount_share', 'mean'),
            mean_coupons_per_100=('coupons_per_100_items', 'mean')
        )
        .round(2)
    )
    
    print("\nSpending & promotion behavior by kids status:")
    display(kids_spend)
    
    heavy_with_kids_pct = kids_promo.loc[1, 'promo_heavy_share'] if 1 in kids_promo.index else 0
    heavy_no_kids_pct = kids_promo.loc[0, 'promo_heavy_share'] if 0 in kids_promo.index else 0
    
    print("\n📊 Key Finding:")
    kids_summary_df = pd.DataFrame({
        'Household Type': ['With Kids', 'Without Kids'],
        'Promo-Heavy Rate': [f'{heavy_with_kids_pct:.1%}', f'{heavy_no_kids_pct:.1%}']
    })
    display(kids_summary_df.set_index('Household Type'))
    
    if heavy_with_kids_pct > heavy_no_kids_pct:
        print("   → Families with children are MORE promotion-aware")
    else:
        print("   → Childless households are MORE promotion-aware")
else:
    print("⚠️ Kids data not available in demographics")



In [ ]:

print("\n" + "=" * 80)
print("DEMOGRAPHIC BREAKDOWN: INCOME")

if 'income' in households.columns:
    income_promo = (
        households[households['income'].notna()]
        .groupby(['income', 'promo_segment'])
        .size()
        .unstack(fill_value=0)
    )
    
    income_promo['total'] = income_promo.sum(axis=1)
    income_promo['promo_heavy_share'] = (
        income_promo['promo_heavy'] / income_promo['total']
    )
    
    income_promo_sorted = income_promo.sort_values('promo_heavy_share', ascending=False)
    
    print("\nPromotion segment distribution by income (sorted by promo-heavy %):")
    display(income_promo_sorted)
    
    income_spend = (
        households[households['income'].notna()]
        .groupby(['income', 'promo_segment'])
        .agg(
            num_households=('household_id', 'nunique'),
            mean_total_sales=('total_sales', 'mean'),
            mean_avg_basket=('avg_basket_value', 'mean'),
            mean_discount_share=('discount_share', 'mean')
        )
        .round(2)
    )
    
    print("\nSpending behavior by income bracket:")
    display(income_spend.sort_index())
    
    top_promo_income = income_promo_sorted.index[0]
    top_promo_pct = income_promo_sorted.iloc[0]['promo_heavy_share']
    
    print("\n📊 Key Finding:")
    income_summary_df = pd.DataFrame({
        'Metric': ['Most Promo-Aware Income Bracket', 'Promo-Heavy Rate'],
        'Value': [top_promo_income, f'{top_promo_pct:.1%}']
    })
    display(income_summary_df.set_index('Metric'))
else:
    print("⚠️ Income data not available in demographics")



In [ ]:
if "income" in households.columns:
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    ax1 = axes[0]
    income_promo_sorted["promo_heavy_share"].plot(kind="barh", ax=ax1, color="#3498db")
    ax1.set_xlabel("% Promo-Heavy", fontsize=12, fontweight="bold")
    ax1.set_ylabel("Income Bracket", fontsize=12, fontweight="bold")
    ax1.set_title("Promotion Usage by Income Level", fontsize=13, fontweight="bold")
    ax1.axvline(
        x=0.25,
        color="red",
        linestyle="--",
        linewidth=2,
        alpha=0.7,
        label="Overall Average (25%)",
    )
    ax1.legend()
    ax1.grid(axis="x", alpha=0.3)

    for i, v in enumerate(income_promo_sorted["promo_heavy_share"]):
        ax1.text(v + 0.01, i, f"{v:.1%}", va="center", fontsize=10, fontweight="bold")

    ax2 = axes[1]

    basket_by_income = households[households["income"].notna()].pivot_table(
        values="avg_basket_value",
        index="income",
        columns="promo_segment",
        aggfunc="mean",
    )

    basket_by_income.plot(kind="barh", ax=ax2, color=["#27ae60", "#e74c3c"])
    ax2.set_xlabel("Average Basket Value ($)", fontsize=12, fontweight="bold")
    ax2.set_ylabel("Income Bracket", fontsize=12, fontweight="bold")
    ax2.set_title(
        "Basket Value by Income & Promo Segment", fontsize=13, fontweight="bold"
    )
    ax2.legend(title="Promo Segment", loc="lower right")
    ax2.grid(axis="x", alpha=0.3)

    plt.tight_layout()
    plt.show()